In [73]:
from trackml.dataset import load_event
from trackml.dataset import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from tqdm import tqdm
from multiprocessing import Pool
from sklearn.neighbors import KDTree

In [74]:
	def extend(submission,hits):

		df = submission.merge(hits,  on=['hit_id'], how='left')
		df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
		df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
		df = df.assign(arctan2 = np.arctan2(df.z, df.r))

		for angle in range(-180,180,1):

		    #print ('\r %f'%angle, end='',flush=True)
		    #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
		    df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

		    min_num_neighbours = len(df1)
		    if min_num_neighbours<4: continue

		    hit_ids = df1.hit_id.values
		    x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
		    r  = (x**2 + y**2)**0.5
		    r  = r/1000
		    a  = np.arctan2(y,x)
		    tree = KDTree(np.column_stack([a,r]), metric='euclidean')

		    track_ids = list(df1.track_id.unique())
		    num_track_ids = len(track_ids)
		    min_length=3

		    for i in range(num_track_ids):
		        p = track_ids[i]
		        if p==0: continue

		        idx = np.where(df1.track_id==p)[0]
		        if len(idx)<min_length: continue

		        if angle>0:
		            idx = idx[np.argsort( z[idx])]
		        else:
		            idx = idx[np.argsort(-z[idx])]


		        ## start and end points  ##
		        idx0,idx1 = idx[0],idx[-1]
		        a0 = a[idx0]
		        a1 = a[idx1]
		        r0 = r[idx0]
		        r1 = r[idx1]

		        da0 = a[idx[1]] - a[idx[0]]  #direction
		        dr0 = r[idx[1]] - r[idx[0]]
		        direction0 = np.arctan2(dr0,da0) 

		        da1 = a[idx[-1]] - a[idx[-2]]
		        dr1 = r[idx[-1]] - r[idx[-2]]
		        direction1 = np.arctan2(dr1,da1) 

	 
		        ## extend start point
		        ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)
		        direction = np.arctan2(r0-r[ns],a0-a[ns])
		        ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]
	   
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

		        ## extend end point
		        ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)

		        direction = np.arctan2(r[ns]-r1,a[ns]-a1)
		        ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 
		        
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
		#print ('\r')
		df = df[['event_id', 'hit_id', 'track_id']]
		return df


In [75]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [76]:
def merge(cl1, cl2, min_cnt): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) &  (d['N2'].values>min_cnt))
    #cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) )

    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 
    return s1

In [77]:
from threading import Lock
global lock
lock = Lock()

In [78]:
def deeper(i):
    #print(i)
    np.random.seed(i)
    #result = p
    #lock.acquire()
    np.random.shuffle(result)
    #lock.release()
    labels = range(result.shape[1])

    for k in [0]:
        for i in range(len(result[:])):
            labels = merge(labels, result[i], k)

        #submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        #print(score_event(truth, submission))
    return labels

In [14]:
for event_id, hits, cells in load_dataset('../storage/track_ml_data/test.zip', parts=['hits', 'cells'] ):
    break

In [15]:
event_id

0

In [46]:
int(os.listdir('../storage/track_ml_data/test')[0].split('event')[1].split('-')[0])

95

In [85]:
#X = pd.DataFrame()
#for event_id, hits, cells in tqdm(load_dataset('../storage/track_ml_data/test.zip', parts=['hits', 'cells'] )):
def foo(f_name):
    #f_name = files[0]
    hits = pd.read_csv('~/storage/track_ml_data/test/'+f_name)
    #print()
    event_id = int(f_name.split('event')[1].split('-')[0])
    result = np.load('predicts/new/event_{}.npy'.format(event_id))

    second = []
    for k in range(10):
        np.random.seed(k)
        #result = p
        #lock.acquire()
        np.random.shuffle(result)
        #lock.release()
        labels = range(result.shape[1])

        for i in range(len(result[:])):
            labels = merge(labels, result[i], 0)
        
        
        
        second += [labels]
#     pool = Pool(processes=8)
#     second = []
#     for i in pool.imap(deeper, range(8)):
#         second += [i]
#     pool.close()
    second = np.array(second)




    result = np.array(second)
    labels = range(result.shape[1])

    for k in [0]:
        for i in range(len(result[:])):
            labels = merge(labels, result[i], k)

        submission = create_one_event_submission(event_id, hits['hit_id'].values, labels)
        a, inv, c = np.unique(submission['track_id'].values, return_inverse=True, return_counts=True)
        submission['track_id'] = inv.astype('int32')
        
        submission = extend(submission,hits)
        submission = extend(submission,hits)
        submission = extend(submission,hits)

        #print(score_event(truth, submission))
    return submission
    #X = pd.concat([X, submission], axis=0)

In [83]:
files[0]

'event000000095-hits.csv'

In [82]:
files = []
for i in os.listdir('../storage/track_ml_data/test'):
    if 'hits' in i:
        files += [i]

In [84]:
len(files)

125

In [86]:
X = pd.DataFrame()

pool = Pool(processes=8)
for i in tqdm(pool.imap(foo, files)):
    X = pd.concat([X, i], axis=0)
pool.close()

125it [30:09, 14.47s/it]


In [87]:
X.sort_values(['event_id', 'hit_id'], inplace=True)

In [88]:
X

,event_id,hit_id,track_id
0,0,1,10826
1,0,2,5334
2,0,3,3172
3,0,4,0
4,0,5,3448
5,0,6,3685
6,0,7,1
7,0,8,3449
8,0,9,4472
9,0,10,4026


In [89]:
X.to_csv('submit.csv', index=False)   

In [90]:
!kaggle competitions submit trackml-particle-identification -f submit.csv -m "new"


Successfully submitted to TrackML Particle Tracking Challenge

In [29]:
X.shape

(13741466, 3)

In [26]:
X

,event_id,hit_id,track_id
0,0,1,828741073
1,0,2,19342604214
2,0,3,357047
3,0,4,45484543
4,0,5,3072771
5,0,6,438082
6,0,7,6
7,0,8,1631974
8,0,9,2733601
9,0,10,438086
